# __Fetch aoe2recs.com match details sequentially__

### __Load libraries__

In [1]:
from pymongo import MongoClient

from tqdm import tqdm
import requests
import json
import pandas as pd
import numpy as np
import os

import multiprocessing
from joblib import Parallel, delayed

cores = os.cpu_count() - 4
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### __Source functions__

In [2]:
%run aoe2rec_api_modules.ipynb

### __Setup database connection__

In [3]:
db_conn = setup_mongo_conn()

### __Fetch match details__

In [4]:
fetch_matches_iterate_aoe2recs(db_conn, collect = 300000, type = 'missing', clean_empty = True, cores = cores)

      16%|█████████▎                                                  | 4256/27243 [05:26<32:29, 11.79it/s]

KeyboardInterrupt: 